## Welcome

This notebook is a small guide to using with Python to access MarkLogic data. 
It comes with a example MarkLogic project you should install first and the rest of this notebook assumes you have done so. The project creates:

a) A database with a REST endpoint on port 8334
b) A small set of JSON sayings by famous people (in JSON) and some photos of them.

We're going to create a small interface that takes a category of saying, e.g. "inspirational" and returns a table of people who said them.

#### If you haven't already done so, deploy the application to your local server by executing the cell below:

In [ ]:
!gradle mlDeploy

#### Once that's done - load the data into the application

In [ ]:
!gradle importData

The main data consists of a set of sayings records in JSON as below:
```
{
"saying": {
    "quote": "Buy the best, and hope it's good enough.", 
    "person": "Albert Etter", 
    "born": "Siloam Springs, Benton, Arkansas, USA", 
    "dob": "1876-09-20", 
    "occupation": "train engineer", 
    "category": ["practical","fun"]
    }
}
```
First goal then is to get a list of all the unique categories. For that we're usinging the 'values' REST endpoint. To make that work, we have already:

- Configured an index on category in the configuration of the [Content database](files/src/main/ml-config/databases/content-database.json).
- Created an Options file [sayings-options](files/src/main/ml-modules/options/sayings-options.xml) file to configure the REST call.

First off we'll load some libraries we're going to need and set up some global variables for the endpoints


In [128]:
import requests
from requests.auth import HTTPDigestAuth
import pandas
import json
from IPython.display import HTML, display 

# Some Global Vars
PORT=8334
HOST="localhost"
USER="sayings-reader"
PASSWORD="sayings-reader"

With those in place, we can call the REST endpoint. We'll use the standard Requests library for this.
For values, we'll get back both category and count, so we'll create a new list with just the values.

In [129]:
def ml_values(attr):
    url = 'http://%s:%s/v1/values/%s?options=sayings-options' % (HOST,PORT,attr)
    headers = {'Accept': 'application/json'}
    result = requests.get(url, headers=headers,auth=HTTPDigestAuth(USER,PASSWORD))
    status = str(result.status_code)
    cats = [] 
    if status == '200':
        v_raw = result.content.decode('utf-8')
        v_json=json.loads(v_raw)['values-response']['distinct-value']
        for row in v_json:
            cats.append(row['_value'])
    else:
        print(status)
        error = result.json()["errorResponse"]["message"]
        print(error)
    return cats

If we call it with 'categories' we'll get all the distinct categories.

In [130]:
ml_values('categories')

['fun',
 'inspirational',
 'political',
 'practical',
 'pratical',
 'religious',
 'spiritual']

Next step is to use the 'search' REST endpoint. This takes a search string and return a set of result for each matching document. 
Using the search options again, we've set the transform-results to raw - giving us the whole document and the searchable-expression to /sayings to make sure we're just looking at the JSON documents.

In [131]:
def ml_search(query):
    url = 'http://%s:%s/v1/search?q=%s&options=sayings-options' % (HOST,PORT,query)
    headers = {'Accept': 'application/json'}
    result = requests.get(url, headers=headers,auth=HTTPDigestAuth(USER,PASSWORD))
    status = str(result.status_code)
    v_json = None 
    if status == '200':
        v_raw = result.content.decode('utf-8')
        v_json=json.loads(v_raw)['results']
    else:
        print(status)
        error = result.json()["errorResponse"]["message"]
        print(error)
    return v_json

Lastly, we create a function to get those results and create a table from them. To avoid clashing with the existing Notebook CSS, we display it isolated in it's own frame.

In [132]:
def get_category_table(query):
    search=ml_search('cat:' + query)
    css = """<style>
#saying {
  font-family: "Trebuchet MS", Arial, Helvetica, sans-serif;
  border-collapse: collapse;
  width: 100%;
}

#saying td, #saying th {
  border: 1px solid #ddd;
  padding: 8px;
}

#saying tr:nth-child(even){background-color: #f2f2f2;}

#saying tr:hover {background-color: #ddd;}

#saying th {
  padding-top: 12px;
  padding-bottom: 12px;
  text-align: left;
  background-color: #4CAF50;
  color: white;
}
</style>"""
    display(HTML(css + 
    '<table id="saying"><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>%s</td><td>%s</td>' % (row['content']['quote'],row['content']['person']) for row in search)
    )),metadata=dict(isolated=True))

Now for the fun bit. Using the Jupyter Widgets, we can create a little UI of our own. 
Interact takes our list, displays it and calls the functions each time it changes.

In [133]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets;
#get the categories
categories = ml_values('categories')
#display
interact(get_table, query=categories);

interactive(children=(Dropdown(description='query', options=('fun', 'inspirational', 'political', 'practical',…